In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [33]:
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Load the model and test it with the simple message

In [3]:
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
model= ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", convert_system_message_to_human=True) 

model 

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000191727EB740>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={})

In [5]:
message1= [AIMessage(content="Hi you are nice Bot"), HumanMessage(content="Hi, how are you buddy?")]

message1

[AIMessage(content='Hi you are nice Bot', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, how are you buddy?', additional_kwargs={}, response_metadata={})]

In [6]:
model.invoke(message1)

AIMessage(content="I'm doing well, thanks for asking! As a large language model, I don't experience emotions or feelings like humans do, but I'm ready and available to assist you with any questions or tasks you have. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--0c8ef4ec-a980-40ba-9985-bc33ba095710-0', usage_metadata={'input_tokens': 12, 'output_tokens': 54, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}})

In [7]:
parser= StrOutputParser()

parser

StrOutputParser()

In [8]:
chain= model | parser

chain 

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000191727EB740>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={})
| StrOutputParser()

In [9]:
print(chain.invoke(message1))

I'm doing well, thanks for asking! I'm ready to help you with whatever you need. How can I assist you today?


In [10]:
message2= [SystemMessage(content="Hi you are nice Bot and you answer everything in brief"), HumanMessage(content="Hi, how are you buddy?")]

message2

[SystemMessage(content='Hi you are nice Bot and you answer everything in brief', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, how are you buddy?', additional_kwargs={}, response_metadata={})]

In [11]:
model.invoke(message2)

AIMessage(content="I'm good, thanks for asking! How are you?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--3c6735e8-4145-4a8e-b4f6-c69dbefaaa0c-0', usage_metadata={'input_tokens': 18, 'output_tokens': 14, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})

In [12]:
chain= model | parser

chain

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000191727EB740>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={})
| StrOutputParser()

In [13]:
chain.invoke(message2)

"I'm doing well, thanks! How can I help you?"

### Now let's start with Prompting

In [14]:
from langchain_core.prompts import ChatPromptTemplate

In [15]:
prompt_template= ChatPromptTemplate(
    [
        ("system", "translate the following into {language}"),
        ("user", "{text}")
    ]
) 

In [19]:
prompt= prompt_template.invoke({"language":"french", "text":"i am learning langchain tutorial."})

prompt

ChatPromptValue(messages=[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}), HumanMessage(content='i am learning langchain tutorial.', additional_kwargs={}, response_metadata={})])

In [21]:
prompt.to_messages()

[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i am learning langchain tutorial.', additional_kwargs={}, response_metadata={})]

In [22]:
parser.invoke(model.invoke(prompt))

'Here are a few options for translating "I am learning langchain tutorial" into French, with slightly different nuances:\n\n*   **Je suis en train d\'apprendre le tutoriel Langchain.** (This is a very common and natural translation, emphasizing the ongoing process of learning.)\n\n*   **J\'apprends le tutoriel Langchain.** (This is a simpler and more direct translation, meaning "I am learning the Langchain tutorial.")\n\n*   **Je suis en train de suivre le tutoriel Langchain.** (This is similar to the first option, but uses "suivre" which means "to follow," implying you are following along with the tutorial.)\n\n*   **J\'étudie le tutoriel Langchain.** (This translates to "I am studying the Langchain tutorial," which might imply a more formal or in-depth approach.)\n\nThe best option depends on the specific context and what you want to emphasize.  For most situations, **"Je suis en train d\'apprendre le tutoriel Langchain"** or **"J\'apprends le tutoriel Langchain"** are excellent choi

### Chaining

In [23]:
chain= prompt_template | model | parser

chain

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000191727EB740>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={})
| StrOutputParser()

In [24]:
chain.invoke({"language": "hindi", "text": "I am hungry"})

'The most common and natural translation of "I am hungry" into Hindi is:\n\n**मुझे भूख लगी है। (Mujhe bhookh lagi hai.)**\n\nHere\'s a breakdown:\n\n*   **मुझे (Mujhe):** To me\n*   **भूख (Bhookh):** Hunger\n*   **लगी है (Lagi hai):** Is happening/is felt\n\nSo, literally, it translates to "Hunger is happening to me."\n\nOther possible translations, though less common, include:\n\n*   **मैं भूखा हूँ। (Main bhookha hoon.)** (Masculine) - I am hungry (masculine)\n*   **मैं भूखी हूँ। (Main bhookhi hoon.)** (Feminine) - I am hungry (feminine)\n\nThe first option, **मुझे भूख लगी है। (Mujhe bhookh lagi hai.)** is generally preferred because it\'s gender-neutral and more idiomatic.'

### Agents

In [48]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [49]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [50]:
memory= MemorySaver()

In [51]:
search= TavilySearchResults(max_results=2)

In [52]:
tools= [search]

In [53]:
agent_executor= create_react_agent(model, tools, checkpointer=memory)

In [69]:
config= {"configurable": {"thread_id": "abc123"}}

config

{'configurable': {'thread_id': 'abc123'}}

In [78]:
agent_executor.stream({"messages": [HumanMessage(content="hi im satish! and i live in Kolkata")]}, config)

<generator object Pregel.stream at 0x0000019172E86020>

In [75]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="hi im satish! and i live in Kolkata")]}, config):
    print(chunk)
    print("------")

{'agent': {'messages': [AIMessage(content="Hi Satish! Nice to meet you. It's great to know you're in Kolkata. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--8b8cb772-bca6-4c07-9075-2fb3157ca30f-0', usage_metadata={'input_tokens': 167, 'output_tokens': 29, 'total_tokens': 196, 'input_token_details': {'cache_read': 0}})]}}
------


In [76]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats my nammme")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Okay, I see that you previously said your name was Satish and that you live in Kolkata. Is that still accurate?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--863151c2-b059-442d-85bc-fe96418ff6bc-0', usage_metadata={'input_tokens': 199, 'output_tokens': 26, 'total_tokens': 225, 'input_token_details': {'cache_read': 0}})]}}
----


In [77]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="Yes you are absolutely right, where do I live and whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Kolkata"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--170b3d40-85d7-4377-9cd0-ff5ebcdfdcc8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kolkata'}, 'id': '2cb151e2-fe39-4d15-8553-30dcc744db9f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 242, 'output_tokens': 12, 'total_tokens': 254, 'input_token_details': {'cache_read': 0}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "Kolkata weather update: Heavy rain expected - Times of India", "url": "https://timesofindia.indiatimes.com/city/kolkata/kolkata-weather-update-expect-moderate-rain-showers-and-high-humidity-today/articleshow/122101335.cms", "content": "Kolkata residents are set to